### Libraries and Data

In [32]:
#!pip install matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=300      

In [33]:
import pandas as pd
import yfinance as yf

In [34]:
df = yf.download(["MCD"],
                 start="2019-01-01",
                 end="2019-12-31",
                progress=False)

In [35]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,175.410004,176.300003,174.169998,176.059998,154.040115,2571500
2019-01-03,175.449997,176.449997,174.410004,174.899994,153.025146,3728200
2019-01-04,176.029999,179.199997,175.690002,178.279999,155.982437,3194300
2019-01-07,178.139999,181.880005,178.139999,180.220001,157.679779,3283000
2019-01-08,181.960007,182.289993,179.910004,180.600006,158.012268,2941400
...,...,...,...,...,...,...
2019-12-23,197.539993,197.539993,196.029999,196.199997,175.804077,2388900
2019-12-24,195.929993,197.139999,195.669998,196.669998,176.225189,1152500
2019-12-26,196.610001,197.160004,196.160004,197.059998,176.574677,1809600


In [36]:
df.Close

Date
2019-01-02    176.059998
2019-01-03    174.899994
2019-01-04    178.279999
2019-01-07    180.220001
2019-01-08    180.600006
                 ...    
2019-12-23    196.199997
2019-12-24    196.669998
2019-12-26    197.059998
2019-12-27    198.169998
2019-12-30    196.910004
Name: Close, Length: 251, dtype: float64

In [37]:
df.to_csv('CEO MCD.csv') 

In [38]:
pd.read_csv('CEO MCD.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,175.410004,176.300003,174.169998,176.059998,154.040115,2571500
1,2019-01-03,175.449997,176.449997,174.410004,174.899994,153.025146,3728200
2,2019-01-04,176.029999,179.199997,175.690002,178.279999,155.982437,3194300
3,2019-01-07,178.139999,181.880005,178.139999,180.220001,157.679779,3283000
4,2019-01-08,181.960007,182.289993,179.910004,180.600006,158.012268,2941400
...,...,...,...,...,...,...,...
246,2019-12-23,197.539993,197.539993,196.029999,196.199997,175.804077,2388900
247,2019-12-24,195.929993,197.139999,195.669998,196.669998,176.225189,1152500
248,2019-12-26,196.610001,197.160004,196.160004,197.059998,176.574677,1809600
249,2019-12-27,197.330002,198.470001,197.179993,198.169998,177.569290,2263900


In [39]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import statsmodels.api as sm
import yfinance as yf
import pandas_datareader as pdr

In [40]:
file = "CEO MCD.csv" # Can paste the full path here in front of the file name
x1 = pd.read_csv(file)

In [56]:
date_MCD = datetime(2019,11,1)   # Find it in the spreadsheet
date_MCD

datetime.datetime(2019, 11, 1, 0, 0)

### Pre-Event

In [62]:
w1_start   = date_MCD - timedelta(73)
w1_end     = date_MCD - timedelta(1)
df1_MCD    = yf.download('MCD', start=w1_start, end=w1_end, progress=False)['Adj Close']
df1_SP1500 = yf.download('^SP1500', start=w1_start, end=w1_end, progress=False)['Adj Close']


print(len(df1_MCD), len(df1_SP1500 ))                       # JMHLY is the code of Meta；SP1500 is the Market Index; Adj Close is adjusted price

51 51


In [63]:
# Calculate the percentage changes
r1_MCD = df1_MCD.pct_change().dropna() * 100
r1_SP1500 = df1_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r1_MCD, r1_SP1500], axis=1).dropna()
r1_MCD_aligned = aligned_data.iloc[:, 0]
r1_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r1_MCD_aligned), len(r1_SP1500_aligned))

# Calculate the covariance and variance
cov1_MCD = np.cov(r1_MCD_aligned, r1_SP1500_aligned)[0, 1]
var1_mkt = np.var(r1_SP1500_aligned)

# Calculate the beta
beta1_MCD = cov1_MCD / var1_mkt
print(beta1_MCD)

50 50
0.6850291110237975


In [64]:
r1_MCD.tail()

Date
2019-10-24   -1.601340
2019-10-25   -0.719328
2019-10-28   -1.454204
2019-10-29    0.438034
2019-10-30    2.216796
Name: Adj Close, dtype: float64

#### Abnormal Returns

In [65]:
abr1_MCD = r1_MCD -  beta1_MCD * r1_SP1500
abr1_MCD.mean()

np.float64(-0.25871810172359183)

### Post-Event

In [67]:
w2_start = date_MCD+timedelta(1)
w2_end = date_MCD+timedelta(77)
df2_MCD = yf.download('MCD', start=w2_start, end=w2_end, progress=False)['Adj Close']
df2_SP1500 = yf.download('^SP1500', start=w2_start, end=w2_end, progress=False)['Adj Close']
print(len(df2_MCD), len(df2_SP1500 ))

51 51


In [68]:
# Calculate the percentage changes
r2_MCD = df2_MCD.pct_change().dropna() * 100
r2_SP1500 = df2_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r2_MCD, r2_SP1500], axis=1).dropna()
r2_MCD_aligned = aligned_data.iloc[:, 0]
r2_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r2_MCD_aligned), len(r2_SP1500_aligned))

# Calculate the covariance and variance
cov2_MCD = np.cov(r2_MCD_aligned, r2_SP1500_aligned)[0, 1]
var2_mkt = np.var(r2_SP1500_aligned)

# Calculate the beta
beta2_MCD = cov2_MCD / var2_mkt
print(beta2_MCD)

50 50
0.6175303824902706


In [69]:
r2_MCD.head()

Date
2019-11-05    1.865786
2019-11-06    1.040668
2019-11-07   -0.566465
2019-11-08    0.274505
2019-11-11   -0.501027
Name: Adj Close, dtype: float64

#### Abnormal Returns

In [70]:
abr2_MCD = r2_MCD -  beta2_MCD * r2_SP1500
abr2_MCD.mean()

np.float64(0.1475585769657006)

#### Abnormal Returns Change

In [71]:
print(f'Abnormal returns change from {abr1_MCD.mean():.2f} to {abr2_MCD.mean():.2f}.')

Abnormal returns change from -0.26 to 0.15.


### Libraries and Data

In [132]:
#!pip install matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=300      

In [133]:
import pandas as pd
import yfinance as yf

In [134]:
df = yf.download(["KELYA"],
                 start="2019-01-01",
                 end="2019-12-31",
                progress=False)

In [135]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,20.270000,20.860001,20.240000,20.650000,19.396973,103800
2019-01-03,20.530001,21.230000,20.270000,20.450001,19.209112,74200
2019-01-04,20.639999,21.299999,20.610001,21.059999,19.782095,84100
2019-01-07,21.049999,21.350000,20.270000,21.209999,19.922991,60700
2019-01-08,21.430000,22.010000,21.320000,22.000000,20.665056,103400
...,...,...,...,...,...,...
2019-12-23,22.910000,22.969999,22.570000,22.790001,21.671843,101300
2019-12-24,22.750000,22.840000,22.580000,22.730000,21.614784,45700
2019-12-26,22.670000,22.879999,22.209999,22.280001,21.186867,60500


In [136]:
df.Close

Date
2019-01-02    20.650000
2019-01-03    20.450001
2019-01-04    21.059999
2019-01-07    21.209999
2019-01-08    22.000000
                ...    
2019-12-23    22.790001
2019-12-24    22.730000
2019-12-26    22.280001
2019-12-27    22.209999
2019-12-30    22.299999
Name: Close, Length: 251, dtype: float64

In [137]:
df.to_csv('CEO KELYA.csv') 

In [138]:
pd.read_csv('CEO KELYA.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,20.270000,20.860001,20.240000,20.650000,19.396973,103800
1,2019-01-03,20.530001,21.230000,20.270000,20.450001,19.209112,74200
2,2019-01-04,20.639999,21.299999,20.610001,21.059999,19.782095,84100
3,2019-01-07,21.049999,21.350000,20.270000,21.209999,19.922991,60700
4,2019-01-08,21.430000,22.010000,21.320000,22.000000,20.665056,103400
...,...,...,...,...,...,...,...
246,2019-12-23,22.910000,22.969999,22.570000,22.790001,21.671843,101300
247,2019-12-24,22.750000,22.840000,22.580000,22.730000,21.614784,45700
248,2019-12-26,22.670000,22.879999,22.209999,22.280001,21.186867,60500
249,2019-12-27,22.340000,22.340000,22.100000,22.209999,21.120295,65400


In [139]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import statsmodels.api as sm
import yfinance as yf
import pandas_datareader as pdr

In [140]:
file = "CEO KELYA.csv" # Can paste the full path here in front of the file name
x1 = pd.read_csv(file)

In [141]:
date_KELYA = datetime(2019,9,30)   # Find it in the spreadsheet
date_KELYA

datetime.datetime(2019, 9, 30, 0, 0)

### Pre-Event

In [142]:
w1_start   = date_KELYA - timedelta(75)
w1_end     = date_KELYA - timedelta(3)
df1_KELYA    = yf.download('KELYA', start=w1_start, end=w1_end, progress=False)['Adj Close']
df1_SP1500 = yf.download('^SP1500', start=w1_start, end=w1_end, progress=False)['Adj Close']


print(len(df1_KELYA), len(df1_SP1500 ))                       # KELYA is the code of Meta；SP1500 is the Market Index; Adj Close is adjusted price

51 51


In [143]:
# Calculate the percentage changes
r1_KELYA = df1_KELYA.pct_change().dropna() * 100
r1_SP1500 = df1_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r1_KELYA, r1_SP1500], axis=1).dropna()
r1_KELYA_aligned = aligned_data.iloc[:, 0]
r1_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r1_KELYA_aligned), len(r1_SP1500_aligned))

# Calculate the covariance and variance
cov1_KELYA = np.cov(r1_KELYA_aligned, r1_SP1500_aligned)[0, 1]
var1_mkt = np.var(r1_SP1500_aligned)

# Calculate the beta
beta1_KELYA = cov1_KELYA / var1_mkt
print(beta1_KELYA)

50 50
0.8870213334919951


In [144]:
r1_KELYA.tail()

Date
2019-09-20    0.205236
2019-09-23   -1.024169
2019-09-24   -0.496676
2019-09-25    0.665559
2019-09-26    1.239649
Name: Adj Close, dtype: float64

#### Abnormal Return

In [145]:
abr1_KELYA = r1_KELYA -  beta1_KELYA * r1_SP1500
abr1_KELYA.mean()

np.float64(-0.12398611850104607)

### Post-Event

In [146]:
w2_start = date_KELYA+timedelta(1)
w2_end = date_KELYA+timedelta(72)
df2_KELYA = yf.download('KELYA', start=w2_start, end=w2_end, progress=False)['Adj Close']
df2_SP1500 = yf.download('^SP1500', start=w2_start, end=w2_end, progress=False)['Adj Close']
print(len(df2_KELYA), len(df2_SP1500 ))

50 50


In [147]:
r2_KELYA = df2_KELYA.pct_change().dropna()*100
r2_SP1500 = df2_SP1500.pct_change().dropna()*100
print(len(r1_KELYA), len(r1_SP1500 ))

50 50


In [148]:
r2_KELYA.head()

Date
2019-10-02   -0.084952
2019-10-03   -1.105431
2019-10-04    1.676690
2019-10-07   -2.283293
2019-10-08   -0.692343
Name: Adj Close, dtype: float64

#### Beta

In [149]:
cov2_KELYA  = np.cov(r2_KELYA, r2_SP1500)[0,1]
var2_mkt  = np.var(r2_SP1500)
beta2_KELYA  = cov2_KELYA/var2_mkt
beta2_KELYA

np.float64(0.8047997947049105)

#### Abnormal Returns

In [150]:
abr2_KELYA = r2_KELYA -  beta2_KELYA * r2_SP1500
abr2_KELYA.mean()

np.float64(-0.24194103336648373)

### Abnormal Returns Change

In [151]:
print(f'Abnormal returns change from {abr1_KELYA.mean():.2f} to {abr2_KELYA.mean():.2f}.')

Abnormal returns change from -0.12 to -0.24.


## HAIN

### Importing libraries

In [121]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import date, datetime, timedelta
import statsmodels.api as sm
import pandas_datareader as pdr
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=300 

### Fetch Data

In [122]:
df = yf.download(["HAIN"],
                 start="2018-01-01",
                 end="2019-12-31",
                progress=False)

In [123]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,42.630001,42.709999,41.880001,41.950001,41.950001,989000
2018-01-03,42.000000,42.020000,40.880001,40.919998,40.919998,757900
2018-01-04,40.910000,40.970001,40.410000,40.599998,40.599998,692600
2018-01-05,39.430000,40.799999,39.049999,40.509998,40.509998,1556200
2018-01-08,40.599998,41.250000,40.580002,41.070000,41.070000,891700
...,...,...,...,...,...,...
2019-12-23,25.799999,26.000000,25.559999,25.780001,25.780001,696200
2019-12-24,25.809999,25.830000,25.590000,25.740000,25.740000,212700
2019-12-26,25.760000,25.770000,25.490000,25.639999,25.639999,487200


In [124]:
df.Close

Date
2018-01-02    41.950001
2018-01-03    40.919998
2018-01-04    40.599998
2018-01-05    40.509998
2018-01-08    41.070000
                ...    
2019-12-23    25.780001
2019-12-24    25.740000
2019-12-26    25.639999
2019-12-27    25.580000
2019-12-30    25.530001
Name: Close, Length: 502, dtype: float64

In [125]:
df.to_csv('CEO HAIN.csv') 

In [126]:
pd.read_csv('CEO HAIN.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-02,42.630001,42.709999,41.880001,41.950001,41.950001,989000
1,2018-01-03,42.000000,42.020000,40.880001,40.919998,40.919998,757900
2,2018-01-04,40.910000,40.970001,40.410000,40.599998,40.599998,692600
3,2018-01-05,39.430000,40.799999,39.049999,40.509998,40.509998,1556200
4,2018-01-08,40.599998,41.250000,40.580002,41.070000,41.070000,891700
...,...,...,...,...,...,...,...
497,2019-12-23,25.799999,26.000000,25.559999,25.780001,25.780001,696200
498,2019-12-24,25.809999,25.830000,25.590000,25.740000,25.740000,212700
499,2019-12-26,25.760000,25.770000,25.490000,25.639999,25.639999,487200
500,2019-12-27,25.740000,25.809999,25.510000,25.580000,25.580000,587800


In [127]:
file = "CEO HAIN.csv" # Can paste the full path here in front of the file name
x1 = pd.read_csv(file)

In [128]:
date_HAIN = datetime(2018,11,5)   # Find it in the spreadsheet
date_HAIN

datetime.datetime(2018, 11, 5, 0, 0)

### Pre-Event

In [129]:
w1_start   = date_HAIN - timedelta(76)
w1_end     = date_HAIN - timedelta(3)
df1_HAIN    = yf.download('HAIN', start=w1_start, end=w1_end, progress=False)['Adj Close']
df1_SP1500 = yf.download('^SP1500', start=w1_start, end=w1_end, progress=False)['Adj Close']


print(len(df1_HAIN), len(df1_SP1500 ))                       # HAIN is the code of Meta；SP1500 is the Market Index; Adj Close is adjusted price

52 51


In [130]:
# Find common dates
common_dates = df1_HAIN.index.intersection(df1_SP1500.index)

# Filter dataframes to common dates
df1_HAIN = df1_HAIN.loc[common_dates]
df1_SP1500 = df1_SP1500.loc[common_dates]

# Print the lengths
print(len(df1_HAIN), len(df1_SP1500))

51 51


In [131]:
r1_HAIN = df1_HAIN.pct_change().dropna()*100
r1_SP1500 = df1_SP1500.pct_change().dropna()*100
print(len(r1_HAIN), len(r1_SP1500 ))

50 50


In [132]:
r1_HAIN.tail()

Date
2018-10-26   -2.067289
2018-10-29    3.311255
2018-10-30    1.121798
2018-10-31   -1.426310
2018-11-01    2.773314
Name: Adj Close, dtype: float64

#### Beta

In [133]:
cov1_HAIN   = np.cov(r1_HAIN, r1_SP1500)[0,1]
var1_mkt  = np.var(r1_SP1500)
beta1_HAIN  = cov1_HAIN/var1_mkt
beta1_HAIN

np.float64(0.3343870410272843)

#### Abnormal Return

In [134]:
abr1_HAIN = r1_HAIN -  beta1_HAIN * r1_SP1500
abr1_HAIN.mean()

np.float64(-0.21427399816090265)

### Post-Event

In [135]:
w2_start = date_HAIN+timedelta(1)
w2_end = date_HAIN+timedelta(84)
df2_HAIN = yf.download('HAIN', start=w2_start, end=w2_end, progress=False)['Adj Close']
df2_SP1500 = yf.download('^SP1500', start=w2_start, end=w2_end, progress=False)['Adj Close']
print(len(df2_HAIN), len(df2_SP1500 ))

54 50


In [136]:
# Align the dataframes by their indices
r2_HAIN = df2_HAIN[df2_HAIN.index.isin(df2_SP1500.index)]
r2_SP1500 = df2_SP1500[df2_SP1500.index.isin(df2_HAIN.index)]

# Calculate percentage change
r2_HAIN = r2_HAIN.pct_change().dropna() * 100
r2_SP1500 = r2_SP1500.pct_change().dropna() * 100

# Ensure the lengths match before any further operations
if len(r2_HAIN) == len(r2_SP1500):
    print("DataFrames are aligned and ready for further operations.")
else:
    print("Mismatch in DataFrame lengths:", len(r2_HAIN), len(r2_SP1500))

DataFrames are aligned and ready for further operations.


In [137]:
r2_HAIN.head()

Date
2018-11-07     0.000000
2018-11-08   -13.853804
2018-11-09    -1.199469
2018-11-13    -0.719424
2018-11-14     1.177537
Name: Adj Close, dtype: float64

#### Beta

In [138]:
cov2_HAIN  = np.cov(r2_HAIN, r2_SP1500)[0,1]
var2_mkt  = np.var(r2_SP1500)
beta2_HAIN  = cov2_HAIN/var2_mkt
beta2_HAIN

np.float64(0.8496636958522443)

#### Abnormal Return

In [139]:
abr2_HAIN = r2_HAIN -  beta2_HAIN * r2_SP1500
abr2_HAIN.mean()

np.float64(-0.6627535902956797)

### Abnormal Returns Change

In [140]:
print(f'Abnormal returns change from {abr1_HAIN.mean():.2f} to {abr2_HAIN.mean():.2f}.')

Abnormal returns change from -0.21 to -0.66.


### Libraries and Data

In [175]:
#!pip install matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=300      

In [176]:
import pandas as pd
import yfinance as yf

In [177]:
df = yf.download(["CCRN"],
                 start="2019-01-01",
                 end="2019-12-31",
                progress=False)

In [178]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,7.24,7.59,7.19,7.50,7.50,665900
2019-01-03,7.48,7.64,7.20,7.47,7.47,296900
2019-01-04,7.53,7.88,7.42,7.87,7.87,252100
2019-01-07,7.87,7.99,7.75,7.99,7.99,200200
2019-01-08,8.04,8.25,7.96,8.25,8.25,173600
...,...,...,...,...,...,...
2019-12-23,11.46,11.69,11.09,11.64,11.64,218700
2019-12-24,11.62,11.66,11.39,11.62,11.62,63900
2019-12-26,11.61,11.71,11.51,11.66,11.66,85500


In [179]:
df.Close

Date
2019-01-02     7.50
2019-01-03     7.47
2019-01-04     7.87
2019-01-07     7.99
2019-01-08     8.25
              ...  
2019-12-23    11.64
2019-12-24    11.62
2019-12-26    11.66
2019-12-27    11.55
2019-12-30    11.51
Name: Close, Length: 251, dtype: float64

In [180]:
df.to_csv('CEO CCRN.csv') 

In [181]:
pd.read_csv('CEO CCRN.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,7.24,7.59,7.19,7.50,7.50,665900
1,2019-01-03,7.48,7.64,7.20,7.47,7.47,296900
2,2019-01-04,7.53,7.88,7.42,7.87,7.87,252100
3,2019-01-07,7.87,7.99,7.75,7.99,7.99,200200
4,2019-01-08,8.04,8.25,7.96,8.25,8.25,173600
...,...,...,...,...,...,...,...
246,2019-12-23,11.46,11.69,11.09,11.64,11.64,218700
247,2019-12-24,11.62,11.66,11.39,11.62,11.62,63900
248,2019-12-26,11.61,11.71,11.51,11.66,11.66,85500
249,2019-12-27,11.69,11.69,11.41,11.55,11.55,94100


In [182]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import statsmodels.api as sm
import yfinance as yf
import pandas_datareader as pdr

In [183]:
file = "CEO CCRN.csv" # Can paste the full path here in front of the file name
x1 = pd.read_csv(file)

### Pre-Event

In [184]:
date_CCRN = datetime(2019,1,16)   # Find it in the spreadsheet
date_CCRN

datetime.datetime(2019, 1, 16, 0, 0)

In [185]:
w1_start   = date_CCRN - timedelta(83)
w1_end     = date_CCRN - timedelta(1)
df1_CCRN    = yf.download('CCRN', start=w1_start, end=w1_end, progress=False)['Adj Close']
df1_SP1500 = yf.download('^SP1500', start=w1_start, end=w1_end, progress=False)['Adj Close']


print(len(df1_CCRN), len(df1_SP1500 ))                       # CCRN is the code of Meta；SP1500 is the Market Index; Adj Close is adjusted price

54 51


In [186]:
# Calculate the percentage changes
r1_CCRN = df1_CCRN.pct_change().dropna() * 100
r1_SP1500 = df1_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r1_CCRN, r1_SP1500], axis=1).dropna()
r1_CCRN_aligned = aligned_data.iloc[:, 0]
r1_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r1_CCRN_aligned), len(r1_SP1500_aligned))

# Calculate the covariance and variance
cov1_CCRN = np.cov(r1_CCRN_aligned, r1_SP1500_aligned)[0, 1]
var1_mkt = np.var(r1_SP1500_aligned)

# Calculate the beta
beta1_CCRN = cov1_CCRN / var1_mkt
print(beta1_CCRN)

50 50
1.320547872481863


In [187]:
r1_CCRN.tail()

Date
2019-01-08    3.254071
2019-01-09    1.575759
2019-01-10    1.670648
2019-01-11    2.934272
2019-01-14    0.342072
Name: Adj Close, dtype: float64

In [188]:
abr1_CCRN = r1_CCRN -  beta1_CCRN * r1_SP1500
abr1_CCRN.mean()

np.float64(0.26176398458711686)

### Post-Event

In [189]:
w2_start = date_CCRN+timedelta(1)
w2_end = date_CCRN+timedelta(83)


In [190]:
# Download the data
df2_CCRN = yf.download('CCRN', start=w2_start, end=w2_end, progress=False)['Adj Close']
df2_SP1500 = yf.download('^SP1500', start=w2_start, end=w2_end, progress=False)['Adj Close']

# Calculate the percentage changes
r2_CCRN = df2_CCRN.pct_change().dropna() * 100
r2_SP1500 = df2_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r2_CCRN, r2_SP1500], axis=1).dropna()
r2_CCRN_aligned = aligned_data.iloc[:, 0]
r2_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r2_CCRN_aligned), len(r2_SP1500_aligned))

# Calculate the covariance and variance
cov2_CCRN = np.cov(r2_CCRN_aligned, r2_SP1500_aligned)[0, 1]
var2_mkt = np.var(r2_SP1500_aligned)

# Calculate the beta
beta2_CCRN = cov2_CCRN / var2_mkt
print(beta2_CCRN)

50 50
1.1236320895982606


#### Abnormal Returns

In [191]:
abr2_CCRN = r2_CCRN -  beta2_CCRN * r2_SP1500
abr2_CCRN.mean()

np.float64(-0.6765667017151066)

### Abnormal Returns Change

In [192]:
print(f'Abnormal returns change from {abr1_CCRN.mean():.2f} to {abr2_CCRN.mean():.2f}.')

Abnormal returns change from 0.26 to -0.68.


### Libraries and Data

In [28]:
!pip install matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=300      

In [29]:
import pandas as pd
import yfinance as yf

In [30]:
df = yf.download(["ASGN", "CASH", "WFC", "CCRN", "MCD", "CASH", "KELYA","ATI","BK", "HAIN"],
                 start="2018-01-01",
                 end="2019-12-31",
                progress=False)



In [31]:
df

Price                      Adj Close                                          \
Ticker                          ASGN        ATI         BK       CASH   CCRN   
Date                                                                           
2018-01-02 00:00:00+00:00  63.360001  25.620001  44.382736  29.702700  12.92   
2018-01-03 00:00:00+00:00  63.110001  26.110001  44.654873  28.980202  12.39   
2018-01-04 00:00:00+00:00  64.190002  26.639999  45.100185  28.691208  12.46   
2018-01-05 00:00:00+00:00  65.250000  26.670000  45.215637  28.948095  12.56   
2018-01-08 00:00:00+00:00  67.940002  27.260000  45.405300  29.333426  12.64   
...                              ...        ...        ...        ...    ...   
2019-12-23 00:00:00+00:00  70.190002  21.139999  43.417084  34.293346  11.64   
2019-12-24 00:00:00+00:00  70.800003  21.000000  43.399853  34.371479  11.62   
2019-12-26 00:00:00+00:00  71.150002  20.830000  43.520531  34.732883  11.66   
2019-12-27 00:00:00+00:00  71.150002  20.990000  43.373974  34.654739  11.55   
2019-12-30 00:00:00+00:00  71.669998  20.520000  43.253311  34.879383  11.51   

Price                                                                   \
Ticker                          HAIN      KELYA         MCD        WFC   
Date                                                                     
2018-01-02 00:00:00+00:00  41.950001  25.408672  147.811920  50.242729   
2018-01-03 00:00:00+00:00  40.919998  25.594273  147.188965  50.629269   
2018-01-04 00:00:00+00:00  40.599998  25.696352  148.221512  51.262550   
2018-01-05 00:00:00+00:00  40.509998  25.816992  148.520172  51.607967   
2018-01-08 00:00:00+00:00  41.070000  25.965471  148.417816  51.024044   
...                              ...        ...         ...        ...   
2019-12-23 00:00:00+00:00  25.780001  21.671841  175.804062  47.377075   
2019-12-24 00:00:00+00:00  25.740000  21.614782  176.225220  47.385883   
2019-12-26 00:00:00+00:00  25.639999  21.186865  176.574661  47.676426   
2019-12-27 00:00:00+00:00  25.580000  21.120296  177.569290  47.473923   
2019-12-30 00:00:00+00:00  25.530001  21.205877  176.440262  47.192181   

Price                          Close  ...       Open  Volume           \
Ticker                          ASGN  ...        WFC    ASGN      ATI   
Date                                  ...                               
2018-01-02 00:00:00+00:00  63.360001  ...  61.040001  338800  2551300   
2018-01-03 00:00:00+00:00  63.110001  ...  61.220001  175500  2195300   
2018-01-04 00:00:00+00:00  64.190002  ...  61.980000  167500  2419800   
2018-01-05 00:00:00+00:00  65.250000  ...  62.759998  289700  1316800   
2018-01-08 00:00:00+00:00  67.940002  ...  62.660000  350000  3890800   
...                              ...  ...        ...     ...      ...   
2019-12-23 00:00:00+00:00  70.190002  ...  53.340000  187600  1313000   
2019-12-24 00:00:00+00:00  70.800003  ...  53.950001  167800   405700   
2019-12-26 00:00:00+00:00  71.150002  ...  54.049999  246000   423200   
2019-12-27 00:00:00+00:00  71.150002  ...  54.290001  255900   747100   
2019-12-30 00:00:00+00:00  71.669998  ...  54.169998  237500   865300   

Price                                                                         \
Ticker                          BK    CASH    CCRN     HAIN   KELYA      MCD   
Date                                                                           
2018-01-02 00:00:00+00:00  3633100  246900  194600   989000  158100  3696900   
2018-01-03 00:00:00+00:00  2409300  313500  273400   757900  140300  3789600   
2018-01-04 00:00:00+00:00  2886500  189900  222300   692600  121300  2756400   
2018-01-05 00:00:00+00:00  2514700  143400  299100  1556200   99300  3737700   
2018-01-08 00:00:00+00:00  3773100  176100  329400   891700  129200  2060800   
...                            ...     ...     ...      ...     ...      ...   
2019-12-23 00:00:00+00:00  2084400  143200  218700   696200  101300  2388900   
2019-12-24 00:00:00

In [32]:
df.Close

Ticker,ASGN,ATI,BK,CASH,CCRN,HAIN,KELYA,MCD,WFC
Date,,,,,,,,,
2018-01-02 00:00:00+00:00,63.360001,25.620001,53.820000,30.833332,12.92,41.950001,27.379999,173.220001,61.090000
2018-01-03 00:00:00+00:00,63.110001,26.110001,54.150002,30.083332,12.39,40.919998,27.580000,172.490005,61.560001
2018-01-04 00:00:00+00:00,64.190002,26.639999,54.689999,29.783333,12.46,40.599998,27.690001,173.699997,62.330002
2018-01-05 00:00:00+00:00,65.250000,26.670000,54.830002,30.049999,12.56,40.509998,27.820000,174.050003,62.750000
2018-01-08 00:00:00+00:00,67.940002,27.260000,55.060001,30.450001,12.64,41.070000,27.980000,173.929993,62.040001
...,...,...,...,...,...,...,...,...,...
2019-12-23 00:00:00+00:00,70.190002,21.139999,50.369999,35.110001,11.64,25.780001,22.790001,196.199997,53.810001
2019-12-24 00:00:00+00:00,70.800003,21.000000,50.349998,35.189999,11.62,25.740000,22.730000,196.669998,53.820000
2019-12-26 00:00:00+00:00,71.150002,20.830000,50.490002,35.560001,11.66,25.639999,22.280001,197.059998,54.150002


In [33]:
df.to_csv('CEO.csv') 

In [34]:
pd.read_csv('CEO.csv')

,Price,Adj Close,Adj Close.1,Adj Close.2,Adj Close.3,Adj Close.4,Adj Close.5,Adj Close.6,Adj Close.7,Adj Close.8,...,Open.8,Volume,Volume.1,Volume.2,Volume.3,Volume.4,Volume.5,Volume.6,Volume.7,Volume.8
0,Ticker,ASGN,ATI,BK,CASH,CCRN,HAIN,KELYA,MCD,WFC,...,WFC,ASGN,ATI,BK,CASH,CCRN,HAIN,KELYA,MCD,WFC
1,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-02 00:00:00+00:00,63.36000061035156,25.6200008392334,44.38273620605469,29.702699661254883,12.920000076293945,41.95000076293945,25.408672332763672,147.81192016601562,50.24272918701172,...,61.040000915527344,338800,2551300,3633100,246900,194600,989000,158100,3696900,13819300
3,2018-01-03 00:00:00+00:00,63.11000061035156,26.110000610351562,44.65487289428711,28.980201721191406,12.390000343322754,40.91999816894531,25.59427261352539,147.18896484375,50.629268646240234,...,61.220001220703125,175500,2195300,2409300,313500,273400,757900,140300,3789600,14203700
4,2018-01-04 00:00:00+00:00,64.19000244140625,26.639999389648438,45.10018539428711,28.691207885742188,12.460000038146973,40.599998474121094,25.696352005004883,148.2215118408203,51.262550354003906,...,61.97999954223633,167500,2419800,2886500,189900,222300,692600,121300,2756400,18740500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,2019-12-23 00:00:00+00:00,70.19000244140625,21.139999389648438,43.417083740234375,34.2933464050293,11.640000343322754,25.780000686645508,21.67184066772461,175.80406188964844,47.3770751953125,...,53.34000015258789,187600,1313000,2084400,143200,218700,696200,101300,2388900,17637700
500,2019-12-24 00:00:00+00:00,70.80000305175781,21.0,43.39985275268555,34.37147903442383,11.619999885559082,25.739999771118164,21.614782333374023,176.2252197265625,47.38588333129883,...,53.95000076293945,167800,405700,763700,47300,63900,212700,45700,1152500,4635500
501,2019-12-26 00:00:00+00:00,71.1500015258789,20.829999923706055,43.520530700683594,34.73288345336914,11.65999984741211,25.639999389648438,21.186864852905273,176.5746612548828,47.67642593383789,...,54.04999923706055,246000,423200,1211700,95500,85500,487200,60500,1809600,12517100
502,2019-12-27 00:00:00+00:00,71.1500015258789,20.989999771118164,43.37397384643555,34.65473937988281,11.550000190734863,25.579999923706055,21.120296478271484,177.5692901611328,47.47392272949219,...,54.290000915527344,255900,747100,1859200,129800,94100,587800,65400,2263900,9892000


In [35]:
TENST = yf.Tickers(["ASGN", "CASH", "WFC", "CCRN", "MCD", "CASH", "KELYA","ATI","BK", "HAIN"])

In [36]:
TENST.history()

[*********************100%***********************]  9 of 9 completed


Price                          Close                                          \
Ticker                          ASGN        ATI         BK       CASH   CCRN   
Date                                                                           
2024-08-26 00:00:00+00:00  95.349998  66.150002  67.000000  66.460762  14.94   
2024-08-27 00:00:00+00:00  95.260002  65.379997  67.209999  66.190964  14.75   
2024-08-28 00:00:00+00:00  95.139999  64.930000  67.000000  66.610657  15.01   
2024-08-29 00:00:00+00:00  96.940002  64.510002  67.250000  66.810501  15.11   
2024-08-30 00:00:00+00:00  96.160004  63.880001  68.220001  68.769051  14.92   
2024-09-03 00:00:00+00:00  92.709999  60.270000  67.889999  68.189484  14.63   
2024-09-04 00:00:00+00:00  93.059998  59.790001  67.739998  67.060318  15.05   
2024-09-05 00:00:00+00:00  92.019997  59.939999  68.000000  67.330116  14.68   
2024-09-06 00:00:00+00:00  89.739998  58.130001  66.410004  67.170242  13.59   
2024-09-09 00:00:00+00:00  89.970001  59.220001  67.720001  67.490005  13.77   
2024-09-10 00:00:00+00:00  91.300003  59.529999  67.510002  66.040001  13.88   
2024-09-11 00:00:00+00:00  89.169998  61.380001  68.099998  65.089996  13.47   
2024-09-12 00:00:00+00:00  89.739998  62.200001  69.019997  65.209999  13.51   
2024-09-13 00:00:00+00:00  92.720001  61.770000  68.699997  66.779999  14.16   
2024-09-16 00:00:00+00:00  93.910004  61.610001  70.529999  68.099998  14.18   
2024-09-17 00:00:00+00:00  92.510002  61.840000  70.809998  67.889999  14.37   
2024-09-18 00:00:00+00:00  92.089996  62.279999  71.040001  67.779999  14.81   
2024-09-19 00:00:00+00:00  93.980003  64.680000  71.720001  69.099998  14.96   
2024-09-20 00:00:00+00:00  93.250000  63.840000  71.820000  67.339996  14.53   
2024-09-23 00:00:00+00:00  94.139999  65.019997  72.139999  66.610001  14.06   

Price                                                             Dividends  \
Ticker                     HAIN      KELYA         MCD        WFC      ASGN   
Date                                                                          
2024-08-26 00:00:00+00:00  6.83  20.910000  287.009888  56.720001       0.0   
2024-08-27 00:00:00+00:00  8.10  20.799999  287.964355  56.259998       0.0   
2024-08-28 00:00:00+00:00  8.26  20.830000  285.389313  57.070000       0.0   
2024-08-29 00:00:00+00:00  7.98  21.080000  286.194611  57.349998       0.0   
2024-08-30 00:00:00+00:00  8.00  21.110001  286.990021  58.470001       0.0   
2024-09-03 00:00:00+00:00  8.32  20.629999  285.519989  58.369999       0.0   
2024-09-04 00:00:00+00:00  8.50  20.740000  288.029999  58.119999       0.0   
2024-09-05 00:00:00+00:00  8.59  20.540001  287.970001  56.860001       0.0   
2024-09-06 00:00:00+00:00  8.47  20.010000  289.510010  54.000000       0.0   
2024-09-09 00:00:00+00:00  8.33  20.180000  291.290009  54.590000       0.0   
2024-09-10 00:00:00+00:00  7.81  20.540001  291.269989  53.950001       0.0   
2024-09-11 00:00:00+00:00  7.89  20.959999  290.230011  53.730000       0.0   
2024-09-12 00:00:00+00:00  8.13  20.650000  292.350006  51.570000       0.0   
2024-09-13 00:00:00+00:00  8.33  21.139999  296.529999  52.779999       0.0   
2024-09-16 00:00:00+00:00  7.98  21.150000  296.690002  53.790001       0.0   
2024-09-17 00:00:00+00:00  8.06  21.350000  293.750000  54.509998       0.0   
2024-09-18 00:00:00+00:00  8.03  21.309999  292.029999  54.259998       0.0   
2024-09-19 00:00:00+00:00  7.87  21.590000  293.850006  55.799999       0.0   
2024-09-20 00:00:00+00:00  7.74  21.180000  296.829987  56.290001       0.0   
2024-09-23 00:00:00+00:00  7.57  20.959999  299.059998  55.110001       0.0   

Price                      ... Stock Splits  Volume                            \
Ticker                     ...          WFC    ASGN      ATI       BK    CASH   
Date                       ...                                                  
2024-08-26 00:00:00+00:00  ...          0.0  165300   502000  2440600  138400   
2024-08-27 00:00:

In [37]:
df.Close      # Choosing a particular column 

Ticker,ASGN,ATI,BK,CASH,CCRN,HAIN,KELYA,MCD,WFC
Date,,,,,,,,,
2018-01-02 00:00:00+00:00,63.360001,25.620001,53.820000,30.833332,12.92,41.950001,27.379999,173.220001,61.090000
2018-01-03 00:00:00+00:00,63.110001,26.110001,54.150002,30.083332,12.39,40.919998,27.580000,172.490005,61.560001
2018-01-04 00:00:00+00:00,64.190002,26.639999,54.689999,29.783333,12.46,40.599998,27.690001,173.699997,62.330002
2018-01-05 00:00:00+00:00,65.250000,26.670000,54.830002,30.049999,12.56,40.509998,27.820000,174.050003,62.750000
2018-01-08 00:00:00+00:00,67.940002,27.260000,55.060001,30.450001,12.64,41.070000,27.980000,173.929993,62.040001
...,...,...,...,...,...,...,...,...,...
2019-12-23 00:00:00+00:00,70.190002,21.139999,50.369999,35.110001,11.64,25.780001,22.790001,196.199997,53.810001
2019-12-24 00:00:00+00:00,70.800003,21.000000,50.349998,35.189999,11.62,25.740000,22.730000,196.669998,53.820000
2019-12-26 00:00:00+00:00,71.150002,20.830000,50.490002,35.560001,11.66,25.639999,22.280001,197.059998,54.150002


In [38]:
!pip install openpyxl

In [39]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import statsmodels.api as sm
import yfinance as yf
import pandas_datareader as pdr

In [40]:
file = "CEO Dismissal Data 2021.02.03.xlsx" # Can paste the full path here in front of the file name
x1 = pd.ExcelFile(file)

### Pre-Event

In [41]:
date_CASH = datetime(2018,10,30)   # Find it in the spreadsheet
date_CASH

datetime.datetime(2018, 10, 30, 0, 0)

In [42]:
w1_start   = date_CASH - timedelta(75)
w1_end     = date_CASH - timedelta(1)
df1_CASH    = yf.download('CASH', start=w1_start, end=w1_end, progress=False)['Adj Close']
df1_SP1500 = yf.download('^SP1500', start=w1_start, end=w1_end, progress=False)['Adj Close']


print(len(df1_CASH), len(df1_SP1500 ))                       # CASH is the code of Meta；SP1500 is the Market Index; Adj Close is adjusted price

51 50


In [43]:
# Calculate the percentage changes
r1_CASH = df1_CASH.pct_change().dropna() * 100
r1_SP1500 = df1_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r1_CASH, r1_SP1500], axis=1).dropna()
r1_CASH_aligned = aligned_data.iloc[:, 0]
r1_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r1_CASH_aligned), len(r1_SP1500_aligned))

# Calculate the covariance and variance
cov1_CASH = np.cov(r1_CASH_aligned, r1_SP1500_aligned)[0, 1]
var1_mkt = np.var(r1_SP1500_aligned)

# Calculate the beta
beta1_CASH = cov1_CASH / var1_mkt
print(beta1_CASH)

49 49
0.34580656247761754


In [44]:
r1_CASH.tail()

Date
2018-10-22   -2.084193
2018-10-23   -0.626047
2018-10-24   -3.443906
2018-10-25    2.435832
2018-10-26    0.594484
Name: Adj Close, dtype: float64

In [45]:
abr1_CASH = r1_CASH -  beta1_CASH * r1_SP1500
abr1_CASH.mean()

np.float64(-0.3183245120947684)

### Post-Event

In [52]:
w2_start = date_CASH+timedelta(1)
w2_end = date_CASH+timedelta(85)
df2_CASH = yf.download('CASH', start=w2_start, end=w2_end, progress=False)['Adj Close']
df2_SP1500 = yf.download('^SP1500', start=w2_start, end=w2_end, progress=False)['Adj Close']
print(len(df2_CASH), len(df2_SP1500 ))

55 51


In [53]:
# Calculate the percentage changes
r2_CASH = df2_CASH.pct_change().dropna() * 100
r2_SP1500 = df2_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r2_CASH, r2_SP1500], axis=1).dropna()
r2_CASH_aligned = aligned_data.iloc[:, 0]
r2_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r2_CASH_aligned), len(r2_SP1500_aligned))

# Calculate the covariance and variance
cov2_CASH = np.cov(r2_CASH_aligned, r2_SP1500_aligned)[0, 1]
var2_mkt = np.var(r2_SP1500_aligned)

# Calculate the beta
beta2_CASH = cov2_CASH / var2_mkt
print(beta2_CASH)

50 50
0.835245456026178


In [54]:
r2_CASH.head()

Date
2018-11-01   -0.950841
2018-11-02    0.039984
2018-11-05   -1.519397
2018-11-06    3.166854
2018-11-07    1.101928
Name: Adj Close, dtype: float64

In [55]:
abr2_CASH = r2_CASH -  beta2_CASH * r2_SP1500
abr2_CASH.mean()

np.float64(-0.2944128254010507)

In [56]:
print(f'Abnormal returns change from {abr1_CASH.mean():.2f} to {abr2_CASH.mean():.2f}.')

Abnormal returns change from -0.32 to -0.29.


### Libraries and Data

In [119]:
#!pip install matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=300      

In [120]:
import pandas as pd
import yfinance as yf

In [121]:
df = yf.download(["BK"],
                 start="2019-01-01",
                 end="2019-12-31",
                progress=False)

In [122]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,46.549999,47.549999,46.410000,47.500000,39.959984,4229500
2019-01-03,47.340000,47.380001,46.490002,46.560001,39.169197,4415700
2019-01-04,47.380001,47.970001,46.910000,47.910000,40.304893,4881500
2019-01-07,47.650002,48.330002,47.259998,47.799999,40.212364,4223100
2019-01-08,48.080002,48.209999,47.020000,47.619999,40.060936,4594500
...,...,...,...,...,...,...
2019-12-23,50.650002,50.700001,50.090000,50.369999,43.417088,2084400
2019-12-24,50.389999,50.470001,50.169998,50.349998,43.399853,763700
2019-12-26,50.459999,50.650002,50.290001,50.490002,43.520527,1211700


In [123]:
df.Close

Date
2019-01-02    47.500000
2019-01-03    46.560001
2019-01-04    47.910000
2019-01-07    47.799999
2019-01-08    47.619999
                ...    
2019-12-23    50.369999
2019-12-24    50.349998
2019-12-26    50.490002
2019-12-27    50.320000
2019-12-30    50.180000
Name: Close, Length: 251, dtype: float64

In [124]:
df.to_csv('CEO BK.csv') 

In [125]:
pd.read_csv('CEO BK.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,46.549999,47.549999,46.410000,47.500000,39.959984,4229500
1,2019-01-03,47.340000,47.380001,46.490002,46.560001,39.169197,4415700
2,2019-01-04,47.380001,47.970001,46.910000,47.910000,40.304893,4881500
3,2019-01-07,47.650002,48.330002,47.259998,47.799999,40.212364,4223100
4,2019-01-08,48.080002,48.209999,47.020000,47.619999,40.060936,4594500
...,...,...,...,...,...,...,...
246,2019-12-23,50.650002,50.700001,50.090000,50.369999,43.417088,2084400
247,2019-12-24,50.389999,50.470001,50.169998,50.349998,43.399853,763700
248,2019-12-26,50.459999,50.650002,50.290001,50.490002,43.520527,1211700
249,2019-12-27,50.529999,50.639999,50.169998,50.320000,43.373993,1859200


In [126]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import statsmodels.api as sm
import yfinance as yf
import pandas_datareader as pdr

In [127]:
file = "CEO BK.csv" # Can paste the full path here in front of the file name
x1 = pd.read_csv(file)

In [128]:
date_BK = datetime(2019,10,20)   # Find it in the spreadsheet
date_BK

datetime.datetime(2019, 10, 20, 0, 0)

### Pre-Event

In [129]:
w1_start   = date_BK - timedelta(74)
w1_end     = date_BK - timedelta(2)
df1_BK    = yf.download('BK', start=w1_start, end=w1_end, progress=False)['Adj Close']
df1_SP1500 = yf.download('^SP1500', start=w1_start, end=w1_end, progress=False)['Adj Close']


print(len(df1_BK), len(df1_SP1500 ))                       # BK is the code of Meta；SP1500 is the Market Index; Adj Close is adjusted price

51 51


In [130]:
r1_BK = df1_BK.pct_change().dropna()*100
r1_SP1500 = df1_SP1500.pct_change().dropna()*100
print(len(r1_BK), len(r1_SP1500 ))

50 50


In [131]:
r1_BK.tail()

Date
2019-10-11    1.425278
2019-10-14   -0.475970
2019-10-15    1.024811
2019-10-16    0.180337
2019-10-17   -0.135007
Name: Adj Close, dtype: float64

#### Beta

In [132]:
cov1_BK   = np.cov(r1_BK, r1_SP1500)[0,1]
var1_mkt  = np.var(r1_SP1500)
beta1_BK  = cov1_BK/var1_mkt
beta1_BK

np.float64(1.1368663923491262)

#### Abnormal Return

In [133]:
abr1_BK = r1_BK -  beta1_BK * r1_SP1500
abr1_BK.mean()

np.float64(-0.09261767639663412)

In [136]:
w2_start = date_BK+timedelta(1)
w2_end = date_BK+timedelta(75)
df2_BK = yf.download('BK', start=w2_start, end=w2_end, progress=False)['Adj Close']
df2_SP1500 = yf.download('^SP1500', start=w2_start, end=w2_end, progress=False)['Adj Close']
print(len(df2_BK), len(df2_SP1500 ))

51 51


In [137]:
# Calculate the percentage changes
r2_BK = df2_BK.pct_change().dropna() * 100
r2_SP1500 = df2_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r2_BK, r2_SP1500], axis=1).dropna()
r2_BK_aligned = aligned_data.iloc[:, 0]
r2_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r2_BK_aligned), len(r2_SP1500_aligned))

# Calculate the covariance and variance
cov2_BK = np.cov(r2_BK_aligned, r2_SP1500_aligned)[0, 1]
var2_mkt = np.var(r2_SP1500_aligned)

# Calculate the beta
beta2_BK = cov2_BK / var2_mkt
print(beta2_BK)

50 50
0.9994778970292769


In [138]:
r2_BK.head()

Date
2019-10-22    1.029101
2019-10-23    1.278741
2019-10-24   -0.962996
2019-10-25    0.891896
2019-10-28    1.099614
Name: Adj Close, dtype: float64

#### Abnormal Returns

In [139]:
abr2_BK = r2_BK -  beta2_BK * r2_SP1500
abr2_BK.mean()

np.float64(0.07828242233324825)

### Abnormal Returns Change

In [140]:
print(f'Abnormal returns change from {abr1_BK.mean():.2f} to {abr2_BK.mean():.2f}.')

Abnormal returns change from -0.09 to 0.08.


### Libraries and Data

In [121]:
#!pip install matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=300      

In [122]:
import pandas as pd
import yfinance as yf

In [123]:
df = yf.download(["ATI"],
                 start="2019-01-01",
                 end="2019-12-31",
                progress=False)

In [124]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,21.240000,22.420000,21.000000,22.350000,22.350000,1434600
2019-01-03,22.320000,22.330000,21.340000,21.430000,21.430000,1155300
2019-01-04,21.850000,23.090000,21.850000,22.610001,22.610001,1746500
2019-01-07,22.740000,23.090000,22.100000,22.600000,22.600000,2013500
2019-01-08,22.700001,23.320000,22.400000,23.299999,23.299999,2153800
...,...,...,...,...,...,...
2019-12-23,20.639999,21.260000,20.559999,21.139999,21.139999,1313000
2019-12-24,21.219999,21.299999,20.910000,21.000000,21.000000,405700
2019-12-26,21.110001,21.260000,20.740000,20.830000,20.830000,423200


In [125]:
df.Close

Date
2019-01-02    22.350000
2019-01-03    21.430000
2019-01-04    22.610001
2019-01-07    22.600000
2019-01-08    23.299999
                ...    
2019-12-23    21.139999
2019-12-24    21.000000
2019-12-26    20.830000
2019-12-27    20.990000
2019-12-30    20.520000
Name: Close, Length: 251, dtype: float64

In [126]:
df.to_csv('CEO ATI.csv') 

In [127]:
pd.read_csv('CEO ATI.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,21.240000,22.420000,21.000000,22.350000,22.350000,1434600
1,2019-01-03,22.320000,22.330000,21.340000,21.430000,21.430000,1155300
2,2019-01-04,21.850000,23.090000,21.850000,22.610001,22.610001,1746500
3,2019-01-07,22.740000,23.090000,22.100000,22.600000,22.600000,2013500
4,2019-01-08,22.700001,23.320000,22.400000,23.299999,23.299999,2153800
...,...,...,...,...,...,...,...
246,2019-12-23,20.639999,21.260000,20.559999,21.139999,21.139999,1313000
247,2019-12-24,21.219999,21.299999,20.910000,21.000000,21.000000,405700
248,2019-12-26,21.110001,21.260000,20.740000,20.830000,20.830000,423200
249,2019-12-27,20.850000,21.250000,20.780001,20.990000,20.990000,747100


In [128]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import statsmodels.api as sm
import yfinance as yf
import pandas_datareader as pdr

In [129]:
file = "CEO ATI.csv" # Can paste the full path here in front of the file name
x1 = pd.read_csv(file)

In [130]:
date_ATI = datetime(2019,1,1)   # Find it in the spreadsheet
date_ATI

datetime.datetime(2019, 1, 1, 0, 0)

### Pre-Event

In [136]:
w1_start   = date_ATI - timedelta(81)
w1_end     = date_ATI - timedelta(3)
df1_ATI    = yf.download('ATI', start=w1_start, end=w1_end, progress=False)['Adj Close']
df1_SP1500 = yf.download('^SP1500', start=w1_start, end=w1_end, progress=False)['Adj Close']


print(len(df1_ATI), len(df1_SP1500 ))                       # ATI is the code of Meta；SP1500 is the Market Index; Adj Close is adjusted price

53 51


In [137]:
# Calculate the percentage changes
r1_ATI = df1_ATI.pct_change().dropna() * 100
r1_SP1500 = df1_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r1_ATI, r1_SP1500], axis=1).dropna()
r1_ATI_aligned = aligned_data.iloc[:, 0]
r1_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r1_ATI_aligned), len(r1_SP1500_aligned))

# Calculate the covariance and variance
cov1_ATI = np.cov(r1_ATI_aligned, r1_SP1500_aligned)[0, 1]
var1_mkt = np.var(r1_SP1500_aligned)

# Calculate the beta
beta1_ATI = cov1_ATI / var1_mkt
print(beta1_ATI)

50 50
1.4511079747435804


In [139]:
r1_ATI.tail()

Date
2018-12-21   -1.181819
2018-12-24   -2.437906
2018-12-26    4.526172
2018-12-27    0.586374
2018-12-28   -1.614344
Name: Adj Close, dtype: float64

#### Abnormal Return

In [140]:
abr1_ATI = r1_ATI -  beta1_ATI * r1_SP1500
abr1_ATI.mean()

np.float64(-0.020827830410049562)

### Post-Event

In [142]:
w2_start = date_ATI+timedelta(3)
w2_end = date_ATI+timedelta(82)
df2_ATI = yf.download('ATI', start=w2_start, end=w2_end, progress=False)['Adj Close']
df2_SP1500 = yf.download('^SP1500', start=w2_start, end=w2_end, progress=False)['Adj Close']
print(len(df2_ATI), len(df2_SP1500 ))

54 51


In [144]:
# Calculate the percentage changes
r2_ATI = df2_ATI.pct_change().dropna() * 100
r2_SP1500 = df2_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r2_ATI, r2_SP1500], axis=1).dropna()
r2_ATI_aligned = aligned_data.iloc[:, 0]
r2_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r2_ATI_aligned), len(r2_SP1500_aligned))

# Calculate the covariance and variance
cov2_ATI = np.cov(r2_ATI_aligned, r2_SP1500_aligned)[0, 1]
var2_mkt = np.var(r2_SP1500_aligned)

# Calculate the beta
beta2_ATI = cov2_ATI / var2_mkt
print(beta2_ATI)

50 50
1.949955509700005


In [145]:
r2_ATI.head()

Date
2019-01-07   -0.044229
2019-01-08    3.097340
2019-01-09    6.266099
2019-01-10   -1.696285
2019-01-11    1.848812
Name: Adj Close, dtype: float64

#### Abnormal Return

In [146]:
abr2_ATI = r2_ATI -  beta2_ATI * r2_SP1500
abr2_ATI.mean()

np.float64(-0.2387207461897456)

### Abnormal Returns Change

In [147]:
print(f'Abnormal returns change from {abr1_ATI.mean():.2f} to {abr2_ATI.mean():.2f}.')

Abnormal returns change from -0.02 to -0.24.


### Libraries and Data

In [34]:
#!pip install matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=300      

In [35]:
import pandas as pd
import yfinance as yf

In [36]:
df = yf.download(["ASGN"],
                 start="2019-01-01",
                 end="2019-12-31",
                progress=False)

In [37]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,53.340000,55.130001,52.770000,54.310001,54.310001,390800
2019-01-03,53.480000,53.639999,51.750000,53.150002,53.150002,446800
2019-01-04,54.209999,56.160000,54.160000,55.910000,55.910000,380900
2019-01-07,55.700001,57.540001,55.000000,57.029999,57.029999,733400
2019-01-08,57.849998,58.939999,57.509998,58.889999,58.889999,317700
...,...,...,...,...,...,...
2019-12-23,70.239998,70.790001,69.099998,70.190002,70.190002,187600
2019-12-24,70.190002,70.989998,69.709999,70.800003,70.800003,167800
2019-12-26,70.790001,71.309998,70.550003,71.150002,71.150002,246000


In [38]:
df.Close

Date
2019-01-02    54.310001
2019-01-03    53.150002
2019-01-04    55.910000
2019-01-07    57.029999
2019-01-08    58.889999
                ...    
2019-12-23    70.190002
2019-12-24    70.800003
2019-12-26    71.150002
2019-12-27    71.150002
2019-12-30    71.669998
Name: Close, Length: 251, dtype: float64

In [39]:
df.to_csv('CEO ASGN.csv') 

In [40]:
pd.read_csv('CEO ASGN.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,53.340000,55.130001,52.770000,54.310001,54.310001,390800
1,2019-01-03,53.480000,53.639999,51.750000,53.150002,53.150002,446800
2,2019-01-04,54.209999,56.160000,54.160000,55.910000,55.910000,380900
3,2019-01-07,55.700001,57.540001,55.000000,57.029999,57.029999,733400
4,2019-01-08,57.849998,58.939999,57.509998,58.889999,58.889999,317700
...,...,...,...,...,...,...,...
246,2019-12-23,70.239998,70.790001,69.099998,70.190002,70.190002,187600
247,2019-12-24,70.190002,70.989998,69.709999,70.800003,70.800003,167800
248,2019-12-26,70.790001,71.309998,70.550003,71.150002,71.150002,246000
249,2019-12-27,71.400002,71.709999,70.870003,71.150002,71.150002,255900


In [41]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import statsmodels.api as sm
import yfinance as yf
import pandas_datareader as pdr

In [42]:
file = "CEO ASGN.csv" # Can paste the full path here in front of the file name
x1 = pd.read_csv(file)

### Pre-Event 

In [43]:
date_ASGN = datetime(2019,4,30)   # Find it in the spreadsheet
date_ASGN

datetime.datetime(2019, 4, 30, 0, 0)

In [47]:
w1_start   = date_ASGN - timedelta(83)
w1_end     = date_ASGN - timedelta(1)
df1_ASGN    = yf.download('ASGN', start=w1_start, end=w1_end, progress=False)['Adj Close']
df1_SP1500 = yf.download('^SP1500', start=w1_start, end=w1_end, progress=False)['Adj Close']


print(len(df1_ASGN), len(df1_SP1500 ))                      

56 51


In [48]:
# Calculate the percentage changes
r1_ASGN = df1_ASGN.pct_change().dropna() * 100
r1_SP1500 = df1_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r1_ASGN, r1_SP1500], axis=1).dropna()
r1_ASGN_aligned = aligned_data.iloc[:, 0]
r1_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r1_ASGN_aligned), len(r1_SP1500_aligned))

# Calculate the covariance and variance
cov1_ASGN = np.cov(r1_ASGN_aligned, r1_SP1500_aligned)[0, 1]
var1_mkt = np.var(r1_SP1500_aligned)

# Calculate the beta
beta1_ASGN = cov1_ASGN / var1_mkt
print(beta1_ASGN)

50 50
2.0481426582516287


In [49]:
r1_ASGN.tail()

Date
2019-04-22   -1.429201
2019-04-23    1.050461
2019-04-24   -0.497810
2019-04-25   -6.945258
2019-04-26    2.166349
Name: Adj Close, dtype: float64

In [50]:
abr1_ASGN = r1_ASGN -  beta1_ASGN * r1_SP1500
abr1_ASGN.mean()

np.float64(-0.3102149975415303)

### Post-Event

In [52]:
w2_start = date_ASGN+timedelta(1)
w2_end = date_ASGN+timedelta(78)
df2_ASGN = yf.download('ASGN', start=w2_start, end=w2_end, progress=False)['Adj Close']
df2_SP1500 = yf.download('^SP1500', start=w2_start, end=w2_end, progress=False)['Adj Close']
print(len(df2_ASGN), len(df2_SP1500 ))

53 51


In [53]:
# Calculate the percentage changes
r2_ASGN = df2_ASGN.pct_change().dropna() * 100
r2_SP1500 = df2_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r2_ASGN, r2_SP1500], axis=1).dropna()
r2_ASGN_aligned = aligned_data.iloc[:, 0]
r2_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r2_ASGN_aligned), len(r2_SP1500_aligned))

# Calculate the covariance and variance
cov2_ASGN = np.cov(r2_ASGN_aligned, r2_SP1500_aligned)[0, 1]
var2_mkt = np.var(r2_SP1500_aligned)

# Calculate the beta
beta2_ASGN = cov2_ASGN / var2_mkt
print(beta2_ASGN)

50 50
1.6260446127708053


In [54]:
abr2_ASGN = r2_ASGN -  beta2_ASGN * r2_SP1500
abr2_ASGN.mean()

np.float64(-0.2292045844925183)

### Abnormal Returns

In [55]:
print(f'Abnormal returns change from {abr1_ASGN.mean():.2f} to {abr2_ASGN.mean():.2f}.')

Abnormal returns change from -0.31 to -0.23.


### Libraries and Data

In [109]:
#!pip install matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=300      

In [110]:
import pandas as pd
import yfinance as yf

In [111]:
df = yf.download(["ACN"],
                 start="2019-01-01",
                 end="2019-12-31",
                progress=False)

In [112]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,138.929993,141.190002,138.279999,140.589996,129.528748,2059300
2019-01-03,139.130005,139.570007,135.580002,135.789993,125.106400,2801300
2019-01-04,138.119995,141.339996,137.500000,141.070007,129.971008,2296400
2019-01-07,140.830002,142.369995,139.850006,141.559998,130.422424,2315600
2019-01-08,144.630005,146.809998,142.690002,145.139999,133.720795,2647800
...,...,...,...,...,...,...
2019-12-23,210.960007,211.289993,208.839996,210.830002,196.692764,2220900
2019-12-24,210.710007,211.669998,210.559998,211.610001,197.420486,998500
2019-12-26,211.610001,212.119995,211.259995,212.050003,197.830963,1059800


In [113]:
df.Close

Date
2019-01-02    140.589996
2019-01-03    135.789993
2019-01-04    141.070007
2019-01-07    141.559998
2019-01-08    145.139999
                 ...    
2019-12-23    210.830002
2019-12-24    211.610001
2019-12-26    212.050003
2019-12-27    212.220001
2019-12-30    210.639999
Name: Close, Length: 251, dtype: float64

In [114]:
df.to_csv('CEO ACN.csv') 

In [115]:
pd.read_csv('CEO ACN.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,138.929993,141.190002,138.279999,140.589996,129.528748,2059300
1,2019-01-03,139.130005,139.570007,135.580002,135.789993,125.106400,2801300
2,2019-01-04,138.119995,141.339996,137.500000,141.070007,129.971008,2296400
3,2019-01-07,140.830002,142.369995,139.850006,141.559998,130.422424,2315600
4,2019-01-08,144.630005,146.809998,142.690002,145.139999,133.720795,2647800
...,...,...,...,...,...,...,...
246,2019-12-23,210.960007,211.289993,208.839996,210.830002,196.692764,2220900
247,2019-12-24,210.710007,211.669998,210.559998,211.610001,197.420486,998500
248,2019-12-26,211.610001,212.119995,211.259995,212.050003,197.830963,1059800
249,2019-12-27,212.449997,212.630005,211.610001,212.220001,197.989578,1292200


In [116]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import statsmodels.api as sm
import yfinance as yf
import pandas_datareader as pdr

In [117]:
file = "CEO ACN.csv" # Can paste the full path here in front of the file name
x1 = pd.read_csv(file)

### Pre-Event

In [118]:
date_ACN = datetime(2019,1,10)   # Find it in the spreadsheet
date_ACN

datetime.datetime(2019, 1, 10, 0, 0)

In [126]:
w1_start   = date_ACN - timedelta(83)
w1_end     = date_ACN - timedelta(1)
df1_ACN    = yf.download('ACN', start=w1_start, end=w1_end, progress=False)['Adj Close']
df1_SP1500 = yf.download('^SP1500', start=w1_start, end=w1_end, progress=False)['Adj Close']


print(len(df1_ACN), len(df1_SP1500 ))                       # ACN is the code of Meta；SP1500 is the Market Index; Adj Close is adjusted price

54 51


In [127]:
# Calculate the percentage changes
r1_ACN = df1_ACN.pct_change().dropna() * 100
r1_SP1500 = df1_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r1_ACN, r1_SP1500], axis=1).dropna()
r1_ACN_aligned = aligned_data.iloc[:, 0]
r1_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r1_ACN_aligned), len(r1_SP1500_aligned))

# Calculate the covariance and variance
cov1_ACN = np.cov(r1_ACN_aligned, r1_SP1500_aligned)[0, 1]
var1_mkt = np.var(r1_SP1500_aligned)

# Calculate the beta
beta1_ACN = cov1_ACN / var1_mkt
print(beta1_ACN)

50 50
1.1167058995602452


In [128]:
r1_ACN.tail()

Date
2019-01-02   -0.297857
2019-01-03   -3.414177
2019-01-04    3.888347
2019-01-07    0.347356
2019-01-08    2.528990
Name: Adj Close, dtype: float64

In [129]:
abr1_ACN = r1_ACN -  beta1_ACN * r1_SP1500
abr1_ACN.mean()

np.float64(0.0646027283836068)

### Post-Event

In [131]:
w2_start = date_ACN+timedelta(1)
w2_end = date_ACN+timedelta(80)
df2_ACN = yf.download('ACN', start=w2_start, end=w2_end, progress=False)['Adj Close']
df2_SP1500 = yf.download('^SP1500', start=w2_start, end=w2_end, progress=False)['Adj Close']
print(len(df2_ACN), len(df2_SP1500 ))

54 51


In [132]:
# Calculate the percentage changes
r2_ACN = df2_ACN.pct_change().dropna() * 100
r2_SP1500 = df2_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r2_ACN, r2_SP1500], axis=1).dropna()
r2_ACN_aligned = aligned_data.iloc[:, 0]
r2_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r2_ACN_aligned), len(r2_SP1500_aligned))

# Calculate the covariance and variance
cov2_ACN = np.cov(r2_ACN_aligned, r2_SP1500_aligned)[0, 1]
var2_mkt = np.var(r2_SP1500_aligned)

# Calculate the beta
beta2_ACN = cov2_ACN / var2_mkt
print(beta2_ACN)

50 50
0.9111389359280789


In [133]:
abr2_ACN = r2_ACN -  beta2_ACN * r2_SP1500
abr2_ACN.mean()

np.float64(0.225808139513773)

In [134]:
print(f'Abnormal returns change from {abr1_ACN.mean():.2f} to {abr2_ACN.mean():.2f}.')

Abnormal returns change from 0.06 to 0.23.


### Libraries and Data

In [51]:
#!pip install matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=300      

In [52]:
import pandas as pd
import yfinance as yf

In [53]:
df = yf.download(["WFC"],
                 start="2019-01-01",
                 end="2019-12-31",
                progress=False)

In [54]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,45.529999,46.950001,45.410000,46.939999,39.733082,20295200
2019-01-03,46.650002,47.349998,46.369999,46.570000,39.419895,22262000
2019-01-04,47.419998,48.110001,47.200001,47.950001,40.588013,23343600
2019-01-07,47.779999,48.230000,47.540001,47.639999,40.325600,21858000
2019-01-08,47.900002,48.160000,46.919998,47.540001,40.240959,19702900
...,...,...,...,...,...,...
2019-12-23,53.340000,54.000000,53.250000,53.810001,47.377083,17637700
2019-12-24,53.950001,54.040001,53.740002,53.820000,47.385876,4635500
2019-12-26,54.049999,54.389999,53.950001,54.150002,47.676426,12517100


In [55]:
df.Close

Date
2019-01-02    46.939999
2019-01-03    46.570000
2019-01-04    47.950001
2019-01-07    47.639999
2019-01-08    47.540001
                ...    
2019-12-23    53.810001
2019-12-24    53.820000
2019-12-26    54.150002
2019-12-27    53.919998
2019-12-30    53.599998
Name: Close, Length: 251, dtype: float64

In [56]:
df.to_csv('CEO WFC.csv') 

In [57]:
pd.read_csv('CEO WFC.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,45.529999,46.950001,45.410000,46.939999,39.733082,20295200
1,2019-01-03,46.650002,47.349998,46.369999,46.570000,39.419895,22262000
2,2019-01-04,47.419998,48.110001,47.200001,47.950001,40.588013,23343600
3,2019-01-07,47.779999,48.230000,47.540001,47.639999,40.325600,21858000
4,2019-01-08,47.900002,48.160000,46.919998,47.540001,40.240959,19702900
...,...,...,...,...,...,...,...
246,2019-12-23,53.340000,54.000000,53.250000,53.810001,47.377083,17637700
247,2019-12-24,53.950001,54.040001,53.740002,53.820000,47.385876,4635500
248,2019-12-26,54.049999,54.389999,53.950001,54.150002,47.676426,12517100
249,2019-12-27,54.290001,54.320000,53.869999,53.919998,47.473927,9892000


In [58]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import statsmodels.api as sm
import yfinance as yf
import pandas_datareader as pdr

In [59]:
file = "CEO WFC.csv" # Can paste the full path here in front of the file name
x1 = pd.read_csv(file)

### Pre-Event

In [60]:
date_WFC = datetime(2019,6,30)   # Find it in the spreadsheet
date_WFC

datetime.datetime(2019, 6, 30, 0, 0)

In [61]:
w1_start   = date_WFC - timedelta(79)
w1_end     = date_WFC - timedelta(1)
df1_WFC    = yf.download('WFC', start=w1_start, end=w1_end, progress=False)['Adj Close']
df1_SP1500 = yf.download('^SP1500', start=w1_start, end=w1_end, progress=False)['Adj Close']


print(len(df1_WFC), len(df1_SP1500 ))                       # JMHLY is the code of Meta；SP1500 is the Market Index; Adj Close is adjusted price

54 51


In [62]:
# Calculate the percentage changes
r1_WFC = df1_WFC.pct_change().dropna() * 100
r1_SP1500 = df1_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r1_WFC, r1_SP1500], axis=1).dropna()
r1_WFC_aligned = aligned_data.iloc[:, 0]
r1_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r1_WFC_aligned), len(r1_SP1500_aligned))

# Calculate the covariance and variance
cov1_WFC = np.cov(r1_WFC_aligned, r1_SP1500_aligned)[0, 1]
var1_mkt = np.var(r1_SP1500_aligned)

# Calculate the beta
beta1_WFC = cov1_WFC / var1_mkt
print(beta1_WFC)

50 50
0.9014929679473136


In [63]:
r1_WFC.tail()

Date
2019-06-24   -1.322257
2019-06-25   -0.280975
2019-06-26   -0.736874
2019-06-27    1.069896
2019-06-28    2.225083
Name: Adj Close, dtype: float64

In [64]:
abr1_WFC = r1_WFC -  beta1_WFC * r1_SP1500
abr1_WFC.mean()

np.float64(-0.03019519671982991)

### Post-Event

In [65]:
w2_start = date_WFC+timedelta(1)
w2_end = date_WFC+timedelta(74)
df2_WFC = yf.download('WFC', start=w2_start, end=w2_end, progress=False)['Adj Close']
df2_SP1500 = yf.download('^SP1500', start=w2_start, end=w2_end, progress=False)['Adj Close']
print(len(df2_WFC), len(df2_SP1500 ))

51 51


In [66]:
# Calculate the percentage changes
r2_WFC = df2_WFC.pct_change().dropna() * 100
r2_SP1500 = df2_SP1500.pct_change().dropna() * 100

# Align the indices
aligned_data = pd.concat([r2_WFC, r2_SP1500], axis=1).dropna()
r2_WFC_aligned = aligned_data.iloc[:, 0]
r2_SP1500_aligned = aligned_data.iloc[:, 1]

# Check the lengths of the aligned series
print(len(r2_WFC_aligned), len(r2_SP1500_aligned))

# Calculate the covariance and variance
cov2_WFC = np.cov(r2_WFC_aligned, r2_SP1500_aligned)[0, 1]
var2_mkt = np.var(r2_SP1500_aligned)

# Calculate the beta
beta2_WFC = cov2_WFC / var2_mkt
print(beta2_WFC)

50 50
1.261925757159589


In [67]:
r2_WFC.head()

Date
2019-07-02   -0.964590
2019-07-03    0.910467
2019-07-05    0.230802
2019-07-08   -0.502414
2019-07-09    0.631166
Name: Adj Close, dtype: float64

In [68]:
abr2_WFC = r2_WFC -  beta2_WFC * r2_SP1500
abr2_WFC.mean()

np.float64(0.04727674910479052)

### Abnormal Returns Change

In [69]:
print(f'Abnormal returns change from {abr1_WFC.mean():.2f} to {abr2_WFC.mean():.2f}.')

Abnormal returns change from -0.03 to 0.05.
